In [0]:
%run /eReport/PkgGeneral/PrcGeneral

In [0]:
import pandas as pd
from datetime import datetime

class PRC_RCD_INDICADOR_ATRASO():
  
  def __init__(self, id_solicitud):
      self.id_solicitud = id_solicitud
      
      #Obteniendo parametros
      parametros = PrcGeneral(id_solicitud = self.id_solicitud).obtenerParametros()
      self.empresa = parametros['empresa']  
      self.fecha_proceso = parametros['fecha_proceso']  
      self.fecha_proceso_str = self.fecha_proceso.strftime('%Y-%m-%d') 
      
      #Fecha de actualizacion
      fecha_actualizacion = datetime.now()
      self.fecha_actualizacion_str = fecha_actualizacion.strftime("%Y-%m-%d %H:%M:%S")
      
      #Conexion
      connection = PrcGeneral().databaseConnect()
      self.connection = connection
      
      cursor = self.connection.cursor()
      self.cursor = cursor
      
  def ProcesaIndicadorAtraso(self):
    try:    
      ActualizaIndicadoAtraso =""";WITH X AS
      (
      SELECT C.ID, A.CLIENTE_PRODUCTO, B.DIAS_ATRASO, A.IMPORTE, ROW_NUMBER() OVER(PARTITION BY A.CLIENTE_PRODUCTO ORDER BY B.DIAS_ATRASO DESC) NUM
      FROM CLIENTE_PRODUCTO_CONCEPTO AS A JOIN CLIENTE_PRODUCTO AS B
      ON A.CLIENTE_PRODUCTO = B.ID
      JOIN CLIENTE_DEUDOR_PRODUCTO CDP
      ON CDP.ID = B.CLIENTE_DEUDOR_PRODUCTO
      JOIN CLIENTE AS C
      ON C.ID = B.CLIENTE
      JOIN PROCESO_PRODUCTO S
      ON S.PROCESO = 3 -- Indicador de Atraso
      AND S.PRODUCTO = CDP.PRODUCTO
      JOIN PROCESO_PRODUCTO_DETALLE PPD
      ON PPD.PROCESO_PRODUCTO = S.ID
      AND PPD.GRUPO_CONCEPTO_SALDO = 1
      AND PPD.CONCEPTO_SALDO = A.CONCEPTO_SALDO
      AND PPD.SITUACION_CREDITO = B.SITUACION_CREDITO
      WHERE C.FECHA_PROCESO = """"'"+ self.fecha_proceso_str +"""'
      ), Y AS (
      SELECT X.ID, X.CLIENTE_PRODUCTO, X.DIAS_ATRASO, X.IMPORTE, 
      (SELECT SUM(I.IMPORTE) FROM X AS I WHERE I.NUM <= X.NUM) AS SALDO_ACUMULADO,
      SUM(IMPORTE) OVER(PARTITION BY X.CLIENTE_PRODUCTO) * 10/100 AS SALDO_TOTAL
      FROM X), T AS (
      SELECT Y.ID, MAX(Y.DIAS_ATRASO) AS DIAS_ATRASO
      FROM Y 
      WHERE Y.SALDO_ACUMULADO >= Y.SALDO_TOTAL
      GROUP BY Y.ID
      )
      UPDATE A
      SET INDICADOR_ATRASO = CASE
                               WHEN T.DIAS_ATRASO = 0 THEN 'A'
                               WHEN T.DIAS_ATRASO BETWEEN 1  AND 8   THEN 'B'
                               WHEN T.DIAS_ATRASO BETWEEN 9  AND 15  THEN 'C'
                               WHEN T.DIAS_ATRASO BETWEEN 16 AND 30  THEN 'D'
                               WHEN T.DIAS_ATRASO BETWEEN 31 AND 60  THEN 'E'
                               WHEN T.DIAS_ATRASO BETWEEN 61 AND 90  THEN 'F'
                               WHEN T.DIAS_ATRASO BETWEEN 91 AND 120 THEN 'G'
                               ELSE 'H'
                             END
      FROM CLIENTE A JOIN T 
      ON A.ID = T.ID
      WHERE A.FECHA_PROCESO = """"'"+ self.fecha_proceso_str +"""'
      """
      self.cursor.execute(ActualizaIndicadoAtraso)
      self.connection.commit()
      print("proceso finalizado")
      
    except Exception as e:    
      print("error: " + str(e))
      
#objeto = PRC_RCD_INDICADOR_ATRASO(1)
#objeto.ProcesaIndicadorAtraso()